In [7]:
!git clone https://github.com/facebookresearch/detectron2.git

Cloning into 'detectron2'...


In [8]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

2.2.2+cpu False


'gcc' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
!git clone https://github.com/facebookresearch/detectron2.git
import os
%pwd
os.chdir('C:\\Users\\DELL\\OneDrive\\Desktop\\MachineLearning\\face-extraction')
cd detectron2
import detectron2
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [1]:
import os

In [2]:
%pwd

'c:\\Users\\DELL\\OneDrive\\Desktop\\MachineLearning\\face-extraction\\rough'

In [3]:
os.chdir('C:\\Users\\DELL\\OneDrive\\Desktop\\MachineLearning\\face-extraction')

In [10]:
cd detectron2

C:\Users\DELL\OneDrive\Desktop\MachineLearning\face-extraction\detectron2\detectron2\detectron2


!pip install -e .

In [ ]:
!pip install 'git+https://github.com/facebookresearch/detectron2.git'

In [11]:
import detectron2

In [12]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
#from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.structures import BoxMode

In [48]:
import cv2

image_path = 'C:\\Users\\DELL\\OneDrive\\Desktop\\MachineLearning\\face-extraction\\images\\output\\detected_face.png'
image = cv2.imread(image_path)

In [49]:
print(image)

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[ 23  37  38]
  [ 23  37  38]
  [ 23  37  38]
  ...
  [ 29  59  33]
  [ 26  56  30]
  [ 24  53  27]]

 [[ 23  37  38]
  [ 23  37  38]
  [ 23  37  38]
  ...
  [ 29  59  33]
  [ 29  59  33]
  [ 28  58  32]]

 ...

 [[106  33  38]
  [107  33  39]
  [108  33  39]
  ...
  [190 150  87]
  [190 151  88]
  [191 151  88]]

 [[156 102  82]
  [154  99  79]
  [151  95  75]
  ...
  [197 158  95]
  [197 157  94]
  [196 157  94]]

 [[192 148 108]
  [190 146 106]
  [188 142 103]
  ...
  [202 162  99]
  [200 160  97]
  [199 160  97]]]


In [50]:
import torch

# Configure the model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available

# Create predictor
predictor = DefaultPredictor(cfg)

[07/14 18:42:19 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...


In [51]:
outputs = predictor(image)
instances = outputs["instances"]
masks = instances.pred_masks.to("cpu").numpy()
classes = instances.pred_classes.to("cpu").numpy()

In [52]:
instances

Instances(num_instances=2, image_height=269, image_width=524, fields=[pred_boxes: Boxes(tensor([[ 89.9992,  24.3309, 410.5103, 264.3335],
        [  0.9245,   2.5910, 239.9322, 264.1505]])), scores: tensor([0.9548, 0.6084]), pred_classes: tensor([0, 0]), pred_masks: tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, Fal

In [53]:
import numpy as np
from detectron2.data import MetadataCatalog

# Verify the class ID for 'person'
person_class_id = MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).thing_classes.index('person')

# Extract masks for 'person'
face_masks = [masks[i] for i in range(len(masks)) if classes[i] == person_class_id]

# Combine the masks if multiple instances are found
combined_mask = np.sum(face_masks, axis=0)

In [54]:
output_image = np.zeros_like(image)
output_image[combined_mask > 0] = image[combined_mask > 0]

# Optionally, add transparency where the mask is not present
output_image = cv2.cvtColor(output_image, cv2.COLOR_BGR2BGRA)
output_image[combined_mask == 0, 3] = 0  # Set alpha channel to 0 for non-mask regions

In [55]:
output_path = 'C:\\Users\\DELL\\OneDrive\\Desktop\\MachineLearning\\face-extraction\\images\\output\\extracted_face.png'
cv2.imwrite(output_path, output_image)

True

In [56]:
from rembg import remove 
from PIL import Image 
  
# Store path of the image in the variable input_path 
input_path =  'C:\\Users\\DELL\\OneDrive\\Desktop\\MachineLearning\\face-extraction\\images\\output\\extracted_face.png' 
  
# Store path of the output image in the variable output_path 
output_path = 'C:\\Users\\DELL\\OneDrive\\Desktop\\MachineLearning\\face-extraction\\images\\output\\transparent_face_trial.png' 
  
# Processing the image 
input = Image.open(input_path) 
  
# Removing the background from the given Image 
output = remove(input) 
  
#Saving the image in the given path 
output.save(output_path) 

In [57]:
# import cv2
# import numpy as np
# import torch
# from detectron2 import model_zoo
# from detectron2.engine import DefaultPredictor
# from detectron2.config import get_cfg
# from detectron2.data import MetadataCatalog

# # Load the generated image
# image_path = 'image001.png'
# image = cv2.imread(image_path)

# # Configure the model
# cfg = get_cfg()
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set threshold for this model
# cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
# cfg.MODEL.DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # Use GPU if available

# # Create predictor
# predictor = DefaultPredictor(cfg)

# # Make prediction
# outputs = predictor(image)

# # Extract instances and masks
# instances = outputs["instances"]
# masks = instances.pred_masks.to("cpu").numpy()

# # Extract face masks (assuming COCO class ID for 'person' is 0)
# face_masks = [masks[i] for i in range(len(masks)) if instances.pred_classes[i] == 0]

# # Apply the masks to the image and save the result
# for i, mask in enumerate(face_masks):
#     # Create an output image with a transparent background
#     output_image = np.zeros_like(image, dtype=np.uint8)
#     output_image[mask] = image[mask]

#     # Save the masked face region
#     cv2.imwrite(f'face_extraction.png', output_image)

# print("Faces extracted and saved successfully.")